In [1]:
import pandas as pd
import numpy as np
import requests
import json
from keys import gkey

In [9]:
file_name = "data/locs.txt"
with open(file_name) as f:
    content = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
content = [x.strip() for x in content] 
print(len(content))

149


In [2]:
file_name = "data/output_tweets/sentiment_scored_english_tweets.csv"
print("Reading " + file_name + "\n");
data = pd.read_csv(file_name)
print(data.shape)

Reading data/output_tweets/sentiment_scored_english_tweets.csv

(85405, 10)


In [3]:
data.head()

,Unnamed: 0,lang,main,published,sentiment,source_location,main_tk,meaningful_main,clean_main_polarity,vader_polarity
0,3,en,@ZHA_News https://youtu.be/UxQmxIm4q1s Rest i...,2016-04-01T02:35:56Z,POSITIVE,Lucerna,"['https://youtu.be/UxQmxIm4q1s', 'Rest', 'in',...","['https://youtu.be/uxqmxim4q1s', 'rest', 'peac...",0.071428,0.8856
1,5,en,Come and join me at our testing Roadshow on Ap...,2016-04-01T14:42:21Z,POSITIVE,Ennetbürgen,"['Come', 'and', 'join', 'me', 'at', 'our', 'te...","['com', 'join', 'test', 'roadshow', 'april', '...",0.071428,0.7644
2,6,en,Bad Apple! pic.twitter.com/Xad2aVOHFd,2016-04-01T21:50:50Z,NEGATIVE,Suica,"['Bad', 'Apple', '!', 'pic.twitter.com/Xad2aVO...","['bad', 'appl', '!', 'pic.twitter.com/xad2avoh...",-0.249994,-0.5848
3,7,en,Number crunching for the past week - 1 new unf...,2016-04-01T11:43:27Z,NEUTRAL,Arth,"['Number', 'crunching', 'for', 'the', 'past', ...","['numb', 'crunch', 'past', 'week', '-', '1', '...",-0.090908,0.0772
4,9,en,Just posted a photo @ Langnau im Emmental http...,2016-04-01T06:31:13Z,NEUTRAL,Langnau,"['Just', 'posted', 'a', 'photo', '@', 'Langnau...","['just', 'post', 'photo', '@', 'langnau', 'im'...",0.000000,0.0000


In [4]:
locations = pd.unique(data.source_location)
print(locations.shape)
locations

(100,)


array(['Lucerna', 'Ennetbürgen', 'Suica', 'Arth', 'Langnau', 'Poschiavo',
       'Laufen', 'Schwyz', 'Herisau', 'Ginevra', 'Affoltern am Albis',
       'Zermatt', 'Confoederatio Helvetica', 'Einsiedeln', 'Bulle', 'Genf',
       'Zuerich', 'Aigle', 'Zurigo', 'Biel/Bienne', 'Uster', 'Monthey',
       'Interlaken', 'Delémont', 'Frauenfeld', 'Aarau', 'İsviçre',
       'Neuchâtel', 'Thun', 'Bellinzona', 'Lucerne', 'Sankt Gallen',
       'Marly', 'Helvetia', 'Svajcarska', 'Chur', 'スイス', 'Brig', 'Baden',
       'Svizzera', 'Biel', 'Lenzburg', 'Geneve', 'Schaffhausen',
       'Fribourg', 'Winterthur', 'St. Gallen', 'Berna', 'Luzern', 'Suiza',
       'CH', 'Swiss', 'Lausanne', 'Basel', 'Bern', 'Zurich', 'Lugano',
       'Genève', 'Geneva', 'Schweiz', 'Zürich', 'Suisse', 'Switzerland',
       'Laufenburg', 'Suïssa', 'Apples', 'Swiss Confederation', 'Conthey',
       'Weinfelden', 'Sarnen', 'Liestal', 'Olten', 'Willisau', 'Bâle',
       'Lauterbrunnen', 'Basilea', 'Bienne', 'Solothurn', 'Hinwil',

In [5]:
# mapping location -> canton using google api
# function taking the name of the place and returning the long address of it
def getAddress(name):
  glink="https://maps.googleapis.com/maps/api/place/textsearch/json?query={}&key={}".format(name + " Switzerland", gkey)
  r = requests.get(glink)
  dic = json.loads(r.text)
  try:
    return dic['results'][0]['formatted_address']
  except:
    return ''

In [6]:
geocodeLink = "https://maps.googleapis.com/maps/api/geocode/json?sensor=true"
from time import sleep

# Get the resulting Json file from the request, and put it in a json dictionary
def getGeocodeJsonDictionary(name):
    link =  u"{}&address=[{}]".format(geocodeLink, name, gkey)
    sleep(0.2) # a workaround for the rate-limit of Google geocode API
    r = requests.get(link)
    dic = json.loads(r.text)
    return dic

# Get the canton from the administrative_area_level_1 field in the dictionary
def getCantonFromGeocodeDictionary(dic):
    canton = ''
    if dic:
        a = dic['results'][0]['address_components']
        canton=[x['short_name'] for x in a if x['types'][0]=='administrative_area_level_1'][0]   
    return canton

# Get the canton by composing the two previous functions
def getCanton(name):
    return getCantonFromGeocodeDictionary(getGeocodeJsonDictionary(name))

In [7]:
addresses = [[a, getAddress(a)] for a in locations]

In [8]:
addresses

[['Lucerna', 'Lucerne, Switzerland'],
 ['Ennetbürgen', 'Ennetbürgen, Switzerland'],
 ['Suica', 'Switzerland'],
 ['Arth', 'Arth, Switzerland'],
 ['Langnau', 'Langnau im Emmental, Switzerland'],
 ['Poschiavo', 'Poschiavo, Switzerland'],
 ['Laufen', '4242 Laufen, Switzerland'],
 ['Schwyz', 'Schwyz, Switzerland'],
 ['Herisau', 'Herisau, Switzerland'],
 ['Ginevra', 'Geneva, Switzerland'],
 ['Affoltern am Albis', 'Affoltern am Albis, Switzerland'],
 ['Zermatt', '3920 Zermatt, Switzerland'],
 ['Confoederatio Helvetica', 'Switzerland'],
 ['Einsiedeln', 'Einsiedeln, Switzerland'],
 ['Bulle', 'Bulle, Switzerland'],
 ['Genf', 'Geneva, Switzerland'],
 ['Zuerich', 'Zürich, Switzerland'],
 ['Aigle', 'Aigle, Switzerland'],
 ['Zurigo', 'Zürich, Switzerland'],
 ['Biel/Bienne', 'Biel/Bienne, Switzerland'],
 ['Uster', 'Uster, Switzerland'],
 ['Monthey', 'Monthey, Switzerland'],
 ['Interlaken', 'Interlaken, Switzerland'],
 ['Delémont', 'Delémont, Switzerland'],
 ['Frauenfeld', 'Frauenfeld, Switzerland'],


In [9]:
def add(a):
    if a[1] == '':
        if a[0] == 'Suisse' :
            val = 'Switzerland'
        else :
            val = a[0]
    else :
        val = a[1]
    return val

addresses = [[a[0], add(a)] for a in addresses]
addresses = [a for a in addresses if a[1] != 'Switzerland']

In [10]:
addresses

[['Lucerna', 'Lucerne, Switzerland'],
 ['Ennetbürgen', 'Ennetbürgen, Switzerland'],
 ['Arth', 'Arth, Switzerland'],
 ['Langnau', 'Langnau im Emmental, Switzerland'],
 ['Poschiavo', 'Poschiavo, Switzerland'],
 ['Laufen', '4242 Laufen, Switzerland'],
 ['Schwyz', 'Schwyz, Switzerland'],
 ['Herisau', 'Herisau, Switzerland'],
 ['Ginevra', 'Geneva, Switzerland'],
 ['Affoltern am Albis', 'Affoltern am Albis, Switzerland'],
 ['Zermatt', '3920 Zermatt, Switzerland'],
 ['Einsiedeln', 'Einsiedeln, Switzerland'],
 ['Bulle', 'Bulle, Switzerland'],
 ['Genf', 'Geneva, Switzerland'],
 ['Zuerich', 'Zürich, Switzerland'],
 ['Aigle', 'Aigle, Switzerland'],
 ['Zurigo', 'Zürich, Switzerland'],
 ['Biel/Bienne', 'Biel/Bienne, Switzerland'],
 ['Uster', 'Uster, Switzerland'],
 ['Monthey', 'Monthey, Switzerland'],
 ['Interlaken', 'Interlaken, Switzerland'],
 ['Delémont', 'Delémont, Switzerland'],
 ['Frauenfeld', 'Frauenfeld, Switzerland'],
 ['Aarau', 'Aarau, Switzerland'],
 ['Neuchâtel', 'Neuchâtel, Switzerland

In [11]:
cantons = [[a[0], getCanton(a[1])] for a in addresses if a[1] != ""]

In [12]:
cantons

[['Lucerna', 'LU'],
 ['Ennetbürgen', 'NW'],
 ['Arth', 'SZ'],
 ['Langnau', 'BE'],
 ['Poschiavo', 'GR'],
 ['Laufen', 'BL'],
 ['Schwyz', 'SZ'],
 ['Herisau', 'AR'],
 ['Ginevra', 'GE'],
 ['Affoltern am Albis', 'ZH'],
 ['Zermatt', 'VS'],
 ['Einsiedeln', 'SZ'],
 ['Bulle', 'FR'],
 ['Genf', 'GE'],
 ['Zuerich', 'ZH'],
 ['Aigle', 'VD'],
 ['Zurigo', 'ZH'],
 ['Biel/Bienne', 'BE'],
 ['Uster', 'ZH'],
 ['Monthey', 'VS'],
 ['Interlaken', 'BE'],
 ['Delémont', 'JU'],
 ['Frauenfeld', 'TG'],
 ['Aarau', 'AG'],
 ['Neuchâtel', 'NE'],
 ['Thun', 'BE'],
 ['Bellinzona', 'TI'],
 ['Lucerne', 'LU'],
 ['Sankt Gallen', 'SG'],
 ['Marly', 'FR'],
 ['Chur', 'GR'],
 ['Brig', 'VS'],
 ['Baden', 'AG'],
 ['Biel', 'BE'],
 ['Lenzburg', 'AG'],
 ['Geneve', 'GE'],
 ['Schaffhausen', 'SH'],
 ['Fribourg', 'FR'],
 ['Winterthur', 'ZH'],
 ['St. Gallen', 'SG'],
 ['Berna', 'BE'],
 ['Luzern', 'LU'],
 ['Lausanne', 'VD'],
 ['Basel', 'BS'],
 ['Bern', 'BE'],
 ['Zurich', 'ZH'],
 ['Lugano', 'TI'],
 ['Genève', 'GE'],
 ['Geneva', 'GE'],
 ['Zürich',

In [13]:
df_cantons = pd.DataFrame(cantons)
df_cantons

,0,1
0,Lucerna,LU
1,Ennetbürgen,NW
2,Arth,SZ
3,Langnau,BE
4,Poschiavo,GR
5,Laufen,BL
6,Schwyz,SZ
7,Herisau,AR
8,Ginevra,GE
9,Affoltern am Albis,ZH


In [19]:
df_cantons.columns = [['Location','ID']]
df_cantons

,Location,ID
0,Lucerna,LU
1,Ennetbürgen,NW
2,Arth,SZ
3,Langnau,BE
4,Poschiavo,GR
5,Laufen,BL
6,Schwyz,SZ
7,Herisau,AR
8,Ginevra,GE
9,Affoltern am Albis,ZH


In [14]:
df_canton_name = pd.read_csv('data/canton_name.csv', sep=',') # Read the canton names

In [15]:
df_canton_name

,ID
0,ZH
1,BE
2,LU
3,UR
4,SZ
5,OW
6,NW
7,GL
8,ZG
9,FR


In [20]:
df = pd.merge(df_canton_name, df_cantons,on = ['ID'], how='outer') # merge the two datasets to have all the cantons

In [21]:
df

,ID,Location
0,ZH,Affoltern am Albis
1,ZH,Zuerich
2,ZH,Zurigo
3,ZH,Uster
4,ZH,Winterthur
5,ZH,Zurich
6,ZH,Zürich
7,ZH,Hinwil
8,ZH,Bülach
9,BE,Langnau


In [22]:
df.to_csv('data/location_to_canton.csv')